In [1]:
# Working, first 650 epochs lr = 1e-4, 650-1450 lr = 5e-5
# training took 24h on a P100 on kaggle, the code was not that optimized and a GAN can't be compiled.
# had trouble with model colapse and had to lower the batch size and keep precision high
# using lighting AI resulted in model collapse every time, I had to revert to pure pytorch for this one.
# used the WGAN architecure
# I am not proud of this model


# next time I would consider an lr scheduler and a lot of other changes 

In [2]:
import torchvision
from pathlib import Path
import matplotlib.pyplot as plt

import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim.lr_scheduler as lr_scheduler


from torch.utils.data import Dataset

from natsort import natsorted

from PIL import Image

import wandb

import datetime

# cuda is available
if torch.cuda.is_available():
    print('cuda is available')
    # get the number of GPUs
    print('number of GPUs:', torch.cuda.device_count())
    # get the name of the GPU
    print('name of the GPU:', torch.cuda.get_device_name(0))
    # get the current device
    print('current device:', torch.cuda.current_device())

    cuda = True

cuda is available
number of GPUs: 1
name of the GPU: NVIDIA GeForce RTX 3080
current device: 0


In [3]:
os.environ["WANDB_API_KEY"] = "47080269e7b1b5a51a89830cb24c495498237e77"
wandb.init(
    # set the wandb project where this run will be logged
    project="JANGAN4",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.000005,
    "architecture": "WGAN",
    "dataset": "FFHQ",
    "epochs": 3000,
    }
)
# wandb.login()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: filip-szczepanski. Use `wandb login --relogin` to force relogin


In [4]:
n_epochs = 3000
batch_size = 2048
lr = 0.000005
n_cpu = os.cpu_count()
latent_dim = 128
img_size = 64
channels = 3
n_critic = 5
clip_value = 0.1
sample_interval = 400

img_shape = (channels, img_size, img_size)

In [5]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )


    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity




In [6]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

In [7]:
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = CustomImageDataset(img_dir="archive", transform=transform)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

In [8]:
def save_checkpoint(generator, discriminator, optimizer_G, optimizer_D, epoch, loss_G, loss_D, save_dir):
    # Create checkpoint directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    checkpoint = {
        'epoch': epoch,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
        'loss_G': loss_G,
        'loss_D': loss_D
    }
    
    # Save with timestamp and epoch number
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    path = os.path.join(save_dir, f'checkpoint_epoch_{epoch}_{timestamp}.pth')
    torch.save(checkpoint, path)
    
    return path



In [9]:
def load_checkpoint(path, generator, discriminator, optimizer_G, optimizer_D):
    checkpoint = torch.load(path)
    
    # Load model states
    generator.load_state_dict(checkpoint['generator_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    
    # Load optimizer states, uncomment if using a lr scheduler

    # optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
    # optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    
    # Return other training info
    return checkpoint['epoch'], checkpoint['loss_G'], checkpoint['loss_D']


In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# JANGAN4/sd9qsjuk/checkpoints/epoch=748-step=3277624.ckpt

# if resuming training 
# generator.load_state_dict(torch.load("2024-11-05_14-41-32/50G.pth", weights_only=True))
# discriminator.load_state_dict(torch.load("2024-11-05_14-41-32/50D.pth", weights_only=False))

if cuda:
    generator.cuda()
    discriminator.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)

# scheduler_G = lr_scheduler.LinearLR(optimizer_G, start_factor=1.0, end_factor=0.3, total_iters=10)
# scheduler_D = lr_scheduler.LinearLR(optimizer_D, start_factor=1.0, end_factor=0.3, total_iters=10)

# might be possible to lower the precision but I chose to have more stability.
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.set_float32_matmul_precision("highest")
# ----------
#  Training
# ----------

# start_epoch, loss_G, loss_D = load_checkpoint(
#     '/kaggle/input/124/pytorch/default/1/checkpoint_epoch_650_20241231_042627.pth',
#     generator,
#     discriminator,
#     optimizer_G,
#     optimizer_D
# )

batches_done = 0
for epoch in range(n_epochs):

    for i, imgs in enumerate(dataloader):

        # if epoch % 190 == 0:
        #     torch.save(generator.state_dict(), f"{epoch}" + "G.pth")
        #     torch.save(discriminator.state_dict(), f"{epoch}"+ "D.pth")


        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        optimizer_D.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_value, clip_value)
        # print(lr)
        # scheduler_G.step()

        # Train the generator every n_critic iterations
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))

            loss_G.backward()
            optimizer_G.step()

            # scheduler_D.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] [Total loss: %f]"
                % (epoch, n_epochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item(), loss_D.item() + loss_G.item())
            )
            

        # if batches_done % sample_interval == 0:
        #     # #save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        #     # save_image(gen_imgs[:25], f"{now_str}/{batches_done}.png", nrow=5, normalize=True)
        #     wandb.log({"examples": [wandb.Image(image) for image in gen_imgs[:15]]})

        wandb.log({"epoch": epoch+650, "loss_D": loss_D, "loss_G": loss_G, "total_loss": loss_G + loss_D})
        batches_done += 1

    if epoch % 5 == 0:
        wandb.log({"examples": [wandb.Image(image) for image in gen_imgs[:10]]})
    # Model saving
    # if epoch % 50 == 0:
    #     save_path = save_checkpoint(
    #         generator,
    #         discriminator,
    #         optimizer_G,
    #         optimizer_D,
    #         epoch+650,
    #         loss_G.item(),
    #         loss_D.item(),
    #         save_dir='/kaggle/working/checkpoints'
    #     )

/tmp/ipykernel_1997/4149428779.py:56: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789220573/work/torch/csrc/tensor/python_tensor.cpp:78.)
  z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))


[Epoch 0/3000] [Batch 0/137] [D loss: 0.053842] [G loss: 0.018178] [Total loss: 0.072020]
[Epoch 0/3000] [Batch 5/137] [D loss: -0.119266] [G loss: 0.019328] [Total loss: -0.099938]
[Epoch 0/3000] [Batch 10/137] [D loss: -0.268144] [G loss: 0.020459] [Total loss: -0.247685]
[Epoch 0/3000] [Batch 15/137] [D loss: -0.389857] [G loss: 0.021716] [Total loss: -0.368141]
[Epoch 0/3000] [Batch 20/137] [D loss: -0.534714] [G loss: 0.023180] [Total loss: -0.511535]
[Epoch 0/3000] [Batch 25/137] [D loss: -0.702864] [G loss: 0.024720] [Total loss: -0.678144]
[Epoch 0/3000] [Batch 30/137] [D loss: -0.866804] [G loss: 0.026452] [Total loss: -0.840351]
[Epoch 0/3000] [Batch 35/137] [D loss: -1.020482] [G loss: 0.028324] [Total loss: -0.992158]
[Epoch 0/3000] [Batch 40/137] [D loss: -1.266758] [G loss: 0.030409] [Total loss: -1.236349]
[Epoch 0/3000] [Batch 45/137] [D loss: -1.373464] [G loss: 0.032645] [Total loss: -1.340819]
[Epoch 0/3000] [Batch 50/137] [D loss: -1.619501] [G loss: 0.035021] [Tota

In [ ]:
wandb.finish()